# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121736e+02     1.543532e+00
 * time: 0.6848011016845703
     1     1.085481e+01     8.940654e-01
 * time: 2.0310211181640625
     2    -1.228226e+01     9.634674e-01
 * time: 2.718388080596924
     3    -3.434843e+01     8.233876e-01
 * time: 3.668174982070923
     4    -4.800090e+01     6.609128e-01
 * time: 4.6324169635772705
     5    -5.680566e+01     2.845932e-01
 * time: 5.593683958053589
     6    -5.945220e+01     2.747412e-01
 * time: 6.2140419483184814
     7    -6.062499e+01     1.031030e-01
 * time: 6.88067102432251
     8    -6.119585e+01     4.246864e-02
 * time: 7.520668983459473
     9    -6.147872e+01     3.586425e-02
 * time: 8.176743030548096
    10    -6.171315e+01     2.937246e-02
 * time: 8.81228494644165
    11    -6.188294e+01     2.458883e-02
 * time: 9.448822021484375
    12    -6.200539e+01     2.401420e-02
 * time: 10.112662076950073
    13    -6.206713e+01     1.945006e-02
 * time: 10.76079797744751
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557706
    AtomicNonlocal      14.8522671
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336821

    total               -62.261666461944
